In [9]:
from vgg19 import VGG19
from keras.preprocessing import image
from imagenet_utils import preprocess_input
from keras.models import Model, load_model
from keras.models import Model

import matplotlib.pyplot as plt
import matplotlib.axis as ax
import cv2

In [10]:
import numpy as np

In [11]:
base_model = load_model('vgg19.h5')

In [12]:
model = Model(input=base_model.input, output=base_model.get_layer('block4_pool').output)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("bl...)`
  """Entry point for launching an IPython kernel.


In [120]:
def repool(path, grayornot=False, normalornot=False, num=1):
    img = image.load_img(path, grayscale=grayornot)
    
    d_img = np.array(img)
    step_i,step_j = d_img.shape[:2]
    step_i /= num
    step_j /= num
    #fmap = np.zeros((1, 100352*num))
    fmap = np.zeros((1, 5000))
    for i in range(num):
        for j in range(num):
            g = d_img[int(0+i*step_i):int(step_i+i*step_i), int(0+j*step_j):int(step_j+j*step_j)]
            g = cv2.resize(g, (224,224))

            #x = image.img_to_array(img)
            #x = np.expand_dims(x, axis=0)
            if grayornot == True:
                img2 = np.zeros_like(g)
                img2 = np.stack([g]*3, axis=2)
                img2[:,:,0] = g[:,:]
                img2[:,:,1] = g[:,:]
                img2[:,:,2] = g[:,:]
                g = img2
            #x = preprocess_input(x)
            if normalornot == True:
                g = [[g/.255]]
            else:
                g = [[g]]
            block4_pool_features = model.predict(g).flatten()
            #fmap += block4_pool_features[:]
            fmap[0][i*5000:(i+1)*5000] = block4_pool_features[-5000:]

    return fmap

In [121]:

real_pool = repool('../../../Testing data/6F-SR/14.JPG', grayornot=False, normalornot=False, num=1).flatten()
bim_pool = repool('../../../Testing data/6F-SR/15.JPG', grayornot=False, normalornot=False, num=1).flatten()
bim_poolf = repool('../../../Testing data/1F/IMG_5504.JPG', grayornot=False, normalornot=False, num=1).flatten()

In [122]:
score = np.dot(real_pool, bim_pool)/(np.linalg.norm(real_pool)*np.linalg.norm(bim_pool))

In [123]:
score

0.35268088802355824

In [124]:
score = np.dot(real_pool, bim_poolf)/(np.linalg.norm(real_pool)*np.linalg.norm(bim_poolf))

In [125]:
score

0.2297897732218752